# Google Drive'ı Bağlamak

Projemize google drive dizinimizin bağlantısını yapıyoruz.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# drive unmount
# drive.flush_and_unmount()

# "**requests**" ve "**datetime**" kütüphanelerini kullanarak verimizi düzenli hale getirdik

Selenium ile internetten veri çekip ve üzerinde işlemler yaparak projemize uygun bir veri haline getiriyoruz.

In [ ]:
import datetime
import requests

class Covid19_Turkey:
    def __init__(self):
        self.Url = "https://covid19.saglik.gov.tr/covid19api?getir="

    @property
    def __AllJson(self):
        self.Json = requests.get(self.Url + "liste").json()

    def __AutoEditJson(self):
        for i in self.Json:
            i["tarih"] = datetime.datetime.strptime(i["tarih"], "%d.%m.%Y").date()
            i["toplam_test"] = int(i["toplam_test"].replace(".", "")) \
                if i["toplam_test"] != "" else 0
            i["toplam_vaka"] = int(i["toplam_vaka"].replace(".", "")) \
                if i["toplam_vaka"] != "" else 0
            i["toplam_vefat"] = int(i["toplam_vefat"].replace(".", "")) \
                if i["toplam_vefat"] != "" else 0
            i["hastalarda_zaturre_oran"] = float(i["hastalarda_zaturre_oran"].replace(",", ".")) \
                if i["hastalarda_zaturre_oran"] != "" else 0
            i["agir_hasta_sayisi"] = int(i["agir_hasta_sayisi"].replace(".", "")) \
                if i["agir_hasta_sayisi"] != "" else 0
            i["toplam_iyilesen"] = int(i["toplam_iyilesen"].replace(".", "")) \
                if i["toplam_iyilesen"] != "" else 0
            i["gunluk_test"] = int(i["gunluk_test"].replace(".", "")) \
                if i["gunluk_test"] != "" else 0
            i["gunluk_vaka"] = int(i["gunluk_vaka"].replace(".", "")) \
                if i["gunluk_vaka"] != "" else 0
            i["gunluk_vefat"] = int(i["gunluk_vefat"].replace(".", "")) \
                if i["gunluk_vefat"] != "" else 0
            i["gunluk_iyilesen"] = int(i["gunluk_iyilesen"].replace(".", "")) \
                if i["gunluk_iyilesen"] != "" else 0

    def GetBackData(self, DateInData):
        LastDay = datetime.datetime.strptime(DateInData, "%Y-%m-%d")
        self.__AllJson
        LastData = datetime.datetime.strptime(self.Json[0]["tarih"], "%d.%m.%Y")
        self.Json = self.Json[:(LastData.date() - LastDay.date()).days]
        self.__AutoEditJson()
        return self.Json

    @property
    def GetAll(self):
        self.__AllJson
        self.Json = self.Json
        self.__AutoEditJson()
        return self.Json

# Pandas ile Dataset Okuma ve Kaydetme

Selenium ile çektiğimiz verileri dataset olarak kaydetmek için pandas kütüphanesini kullanarak kaydediyoruz.

In [ ]:
import pandas as pd
from os import path


def RenameDropSort(DF):
    return DF.rename(columns={"tarih": "Date",
                              "toplam_test": "Tests",
                              "toplam_vaka": "Confirmed",
                              "toplam_vefat": "Deaths",
                              "hastalarda_zaturre_oran": "Pneumonia",
                              "agir_hasta_sayisi": "SeriouslyIll",
                              "toplam_iyilesen": "Recovered",
                              "gunluk_test": "DailyTests",
                              "gunluk_vaka": "DailyConfirmed",
                              "gunluk_vefat": "DailyDeaths",
                              "gunluk_iyilesen": "DailyRecovered"}).drop(
        columns=["toplam_yogun_bakim", "toplam_entube", "yatak_doluluk_orani",
                 "eriskin_yogun_bakim_doluluk_orani", "ventilator_doluluk_orani",
                 "ortalama_filyasyon_suresi", "ortalama_temasli_tespit_suresi",
                 "filyasyon_orani"]).sort_values(by=["Date"])


Data = Covid19_Turkey()
File = "./DataSets/Covid19-Turkey.csv"
if path.exists(File):
    DF = pd.read_csv(File)
    try:
        DF.to_csv(File + ".backup", index=False)
        DF = DF.append(RenameDropSort(pd.DataFrame(Data.GetBackData(DF["Date"][DF["Date"].count() - 1]))),
                       ignore_index=True)
        print("Saved data.")
    except KeyError:
        print("You have all data.")
else:
    DF = RenameDropSort(pd.DataFrame(Data.GetAll))
    print("File created and saved data.")
DF.to_csv(File, index=False)